In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
from intern.remote.boss import BossRemote
from intern.resource.boss.resource import ChannelResource
import sys
sys.path.append('.')
from NDResource import get_host_token, NeuroDataResource
from tqdm import tqdm

In [2]:
def block_compute(x_start, x_stop,
                  y_start, y_stop,
                  z_start, z_stop,
                  origin=(0, 0, 0),
                  block_size=(512, 512, 16)):
    """
    Get bounding box coordinates (in 3D) of small cutouts to request in
    order to reconstitute a larger cutout.
    Arguments:
        x_start (int): The lower bound of dimension x
        x_stop (int): The upper bound of dimension x
        y_start (int): The lower bound of dimension y
        y_stop (int): The upper bound of dimension y
        z_start (int): The lower bound of dimension z
        z_stop (int): The upper bound of dimension z
    Returns:
        [((x_start, x_stop), (y_start, y_stop), (z_start, z_stop)), ... ]
    """
    # x
    x_bounds = range(origin[0], x_stop + block_size[0], block_size[0])
    x_bounds = [x for x in x_bounds if (x > x_start and x < x_stop)]
    if len(x_bounds) is 0:
        x_slices = [(x_start, x_stop)]
    else:
        x_slices = []
        for start_x in x_bounds[:-1]:
            x_slices.append((start_x, start_x + block_size[0]))
        x_slices.append((x_start, x_bounds[0]))
        x_slices.append((x_bounds[-1], x_stop))

    # y
    y_bounds = range(origin[1], y_stop + block_size[1], block_size[1])
    y_bounds = [y for y in y_bounds if (y > y_start and y < y_stop)]
    if len(y_bounds) is 0:
        y_slices = [(y_start, y_stop)]
    else:
        y_slices = []
        for start_y in y_bounds[:-1]:
            y_slices.append((start_y, start_y + block_size[1]))
        y_slices.append((y_start, y_bounds[0]))
        y_slices.append((y_bounds[-1], y_stop))

    # z
    z_bounds = range(origin[2], z_stop + block_size[2], block_size[2])
    z_bounds = [z for z in z_bounds if (z > z_start and z < z_stop)]
    if len(z_bounds) is 0:
        z_slices = [(z_start, z_stop)]
    else:
        z_slices = []
        for start_z in z_bounds[:-1]:
            z_slices.append((start_z, start_z + block_size[2]))
        z_slices.append((z_start, z_bounds[0]))
        z_slices.append((z_bounds[-1], z_stop))

    # alright, yuck. but now we have {x, y, z}_slices, each of which hold the
    # start- and end-points of each cube-aligned boundary. For instance, if you
    # requested z-slices 4 through 20, it holds [(4, 16), (16, 20)].

    # For my next trick, I'll convert these to a list of:
    # ((x_start, x_stop), (y_start, y_stop), (z_start, z_stop))

    chunks = []
    for x in x_slices:
        for y in y_slices:
            for z in z_slices:
                chunks.append((x, y, z))
    return chunks

## load data

In [3]:
jf = "m247514_Site3Annotation_MN_global.json"

In [4]:
with open("ForrestAnnotationAndCode/m247514_Site3Annotation_MN_global.json") as jd:
    d = json.load(jd)

# Looking at the first set of points

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

In [6]:
print('d.keys():', list(d.keys()))
print(len(d['area_lists']))
print("d['area_lists'].keys():", list(d['area_lists'][0].keys()))
print(len(d['area_lists'][3]['areas']))

d.keys(): ['area_lists']
1046
d['area_lists'].keys(): ['oid', 'id', 'areas']
12


In [7]:
print(d['area_lists'][0].keys())
#print(d['area_lists'][0]['areas'])#a list of dicts
print(d['area_lists'][0]['areas'][0].keys())

dict_keys(['oid', 'id', 'areas'])
dict_keys(['local_path', 'z', 'global_path', 'tileIds'])


In [8]:
def gen_xy_ints(al, i):
    x = [int(i[0]) for i in al['areas'][i]['global_path']]
    y = [int(i[1]) for i in al['areas'][i]['global_path']]
    z = al['areas'][i]['z'] #already int?
    return x,y,z

In [9]:
def give_x_y_get_newx_newy(x,y):
    def points_along_line(x,y):
        #print('NEW SHIT')
        #print(len(x)==len(y))
        if x[1]-x[0]!=0 and y[1]-y[0]==0:
            interp_x = np.linspace(x[0],x[1],np.abs(x[1]-x[0])+1)
            interp_y = [y[0]]*len(interp_x)
        elif y[1]-y[0]!=0 and x[1]-x[0]==0:
            interp_y = np.linspace(y[0],y[1],np.abs(y[1]-y[0])+1)
            interp_x = [x[0]]*len(interp_y)
        elif y[1]-y[0]!=0 and y[1]-y[0]!=0:
            interp_x = np.linspace(x[0],x[1],np.abs(x[1]-x[0])+1)
            interp_y = np.linspace(y[0],y[1],np.abs(y[1]-y[0])+1)
        else:
            interp_x = x
            interp_y = y
        if len(interp_x) < len(interp_y):
            diff = len(interp_y) - len(interp_x)
            interp_x = np.concatenate((interp_x,[interp_x[-1] for a in range(0,diff)]))
        elif len(interp_x) > len(interp_y):
            diff = len(interp_x) - len(interp_y)
            interp_y = np.concatenate((interp_y,[interp_y[-1] for a in range(0,diff)]))
        return interp_x, interp_y
    new_x = []
    new_y = []
    for j in range(-1, len(x)-1):
        interp_x , interp_y = points_along_line([x[j],x[j+1]], [y[j],y[j+1]])
        new_x = new_x + [int(a) for a in interp_x[1:]]
        new_y = new_y + [int(a) for a in interp_y[1:]]
    #plt.scatter(x,y)
    #plt.show()
    #plt.scatter(new_x, new_y)
    #plt.show()
    return new_x, new_y

In [10]:
def give_connected_scatter_get_filled(new_x, new_y, return_coords=True):
    #go from the points to a mask
    x_len = np.max(new_x)-np.min(new_x)+1
    y_len = np.max(new_y)-np.min(new_y)+1
    mask = np.zeros((x_len, y_len))
    mask_x = new_x-np.min(new_x)
    mask_y = new_y-np.min(new_y)
    #print(len(new_x)==len(new_y))
    for i in range(0,len(new_x)):
        mask[mask_x[i]][mask_y[i]] = 1
    #plt.imshow(mask)
    #plt.show()
    #now have mask
    mask_topdown = mask.copy()
    mask_botup = mask.copy()
    counter_down = np.zeros(len(mask))
    for row in range(1,len(mask)-1): #this is inefficient
        for idx in range(0,len(mask[row])-1):
            if mask_topdown[row-1][idx] == 1:
                mask_topdown[row][idx] = 1
    for row in range(len(mask)-2,0,-1): #this is inefficient
        for idx in range(0,len(mask[row])-1):
            if mask_botup[row+1][idx] == 1:
                mask_botup[row][idx] = 1
    ###########
    mask_leftr = mask.T.copy()
    mask_rightl = mask.T.copy()
    for row in range(1,len(mask.T)-1): #this is inefficient
        for idx in range(0,len(mask.T[row])-1):
            if mask_leftr[row-1][idx] == 1:
                mask_leftr[row][idx] = 1
    for row in range(len(mask.T)-2,0,-1): #this is inefficient
        for idx in range(0,len(mask.T[row])-1):
            if mask_rightl[row+1][idx] == 1:
                mask_rightl[row][idx] = 1
    mask3 = np.logical_and(mask_leftr, mask_rightl)
    ###########
    mask2 = np.logical_and(mask_topdown, mask_botup)
    mask4 = np.logical_and(mask2,mask3.T)
    #plt.imshow(mask4)#2
    #plt.show()
    if return_coords:
        #go from the mask back to the points
        final_coords = np.nonzero(mask4)
        final_x = final_coords[0]+np.min(new_x)
        final_y = final_coords[1]+np.min(new_y)
        return final_x, final_y
    else:
        return mask4#2

In [11]:
#fill connected scatter for a single area list
al0 = d['area_lists'][3]#some specific arealist
data = []
for i in range(len(al0['areas'])):
    i=8
    x,y,z = gen_xy_ints(al0, i)
    new_x, new_y = give_x_y_get_newx_newy(x,y)
    final_x, final_y = give_connected_scatter_get_filled(new_x,new_y)
    trace = go.Scatter3d(
        x=final_x, y=final_y, z=[z]*len(final_x),
        mode='markers',
        marker=dict(
            size=1,
            color=z,
            colorscale='Viridis',
            opacity=0.5
            )
    )
    data.append(trace)
    break
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')
    

In [12]:
#scatter for all area_lists
data_trace = []
alcounter = 0
for arealist in d['area_lists'][:5]:
    alcounter += 1
    for i in range(len(arealist['areas'])):
        x,y,z = gen_xy_ints(arealist, i)
        new_x, new_y = give_x_y_get_newx_newy(x,y)
        final_x, final_y = give_connected_scatter_get_filled(new_x,new_y)
        
        trace = go.Scatter3d(
            x=final_x, y=final_y, z=[z]*len(final_x),
            mode ='markers',
            marker = dict(
                size=2,
                color=z,
                colorscale='Viridis',
                opacity=0.2
                )
        )
        data_trace.append(trace)
    print(alcounter, '/50', end="\r")
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data_trace, layout=layout)
py.iplot(fig, filename='0-49')

The draw time for this plot will be slow for clients without much RAM.


/usr/local/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



## Putting stuff into one big tiff:

In [13]:
#meshses for all area_lists
data = []
alcounter = 0
uid = 0
for arealist in d['area_lists']:
    alcounter += 1
    uid +=1
    for i in range(len(arealist['areas'])):
        x,y,z = gen_xy_ints(arealist, i)
        new_x, new_y = give_x_y_get_newx_newy(x,y)
        final_x, final_y = give_connected_scatter_get_filled(new_x,new_y)
        region_data = {'x':final_x, 'y':final_y, 'z':z , 'uid':uid}
        data.append(region_data)
    print(alcounter, '/1046', end="\r")

In [14]:
sorted_z = sorted(data, key=lambda k: k['z'])
tot_z = [int(a['z']) for a in sorted_z]
list_z = []
for val in range(min(tot_z), max(tot_z)+1):
    curr_z = []
    for b in data:
        if int(b['z'])==val:
            curr_z.append(b)
    list_z.append(curr_z)
print(np.sum([len(i) for i in list_z]))
print('each index is z-slice in ascending oder, size is how many arealists on each z:')
print([len(a) for a in list_z])
print("within each arealist, we have a list of dictionaries(areas) with 'x', 'y', 'z'")
print('arealist 1, area 6: ', list_z[0][5])

4945
each index is z-slice in ascending oder, size is how many arealists on each z:
[93, 101, 111, 96, 103, 93, 91, 89, 95, 95, 102, 110, 95, 103, 97, 94, 116, 122, 124, 100, 93, 99, 96, 89, 80, 90, 89, 98, 91, 93, 106, 106, 115, 120, 101, 107, 103, 106, 110, 108, 105, 104, 89, 88, 88, 89, 86, 94, 89, 83]
within each arealist, we have a list of dictionaries(areas) with 'x', 'y', 'z'
arealist 1, area 6:  {'x': array([6870, 6870, 6870, ..., 6881, 6881, 6881]), 'y': array([5849, 5850, 5851, ..., 5953, 5954, 5955]), 'z': 0.0, 'uid': 56}


## Boss Stuff

In [15]:
#we look at https://ndwebtools.neurodata.io/cutout/collman/M247514_Rorb_1_Site3Align2_EM/
voxel_unit = 'nanometers'
x_start = 0
x_stop = 14020
x_voxel_size = 3.0
y_start = 0
y_stop = 14723
y_voxel_size = 3.0
z_start = 0
z_stop = 49
z_voxel_size = 50.0

masks = []
for z_value in list_z:
    mask = np.zeros((x_stop-x_start,y_stop-y_start))
    for arealist in z_value:
        for i in range(0,len(arealist['x'])-1):
            mask[arealist['x'][i]][arealist['y'][i]] = arealist['uid']
    masks.append(mask)
print('number of slices: ', len(masks))
print('each slice: ', mask.shape)

number of slices:  50
each slice:  (14020, 14723)


In [16]:
rmt = BossRemote('./neurodata.cfg')

COLL_NAME = 'collman'
EXP_NAME = 'M247514_Rorb_1_Site3Align2_EM'
CHAN_NAME = 'm247514_Site3Annotation_MN_global'

# Create a new channel that uses uint16 data.
chan_setup = ChannelResource(CHAN_NAME, COLL_NAME, EXP_NAME, 'annotation', datatype='uint64')
chan = rmt.get_project(chan_setup)

# Ranges use the Python convention where the second number is the stop
# value.  Thus, x_rng specifies x values where: 0 <= x < 8.
x_rng = [0, 14020]
y_rng = [0, 14723]
z_rng = [0, 49]

ranges = block_compute(*x_rng,*y_rng,*z_rng)
print(ranges[0])

((512, 1024), (512, 1024), (16, 32))


In [ ]:
for some_range in tqdm(ranges):
    x,y,z = some_range
    data_cut = np.stack([np.transpose(np.array(masks[z_idx][x[0]:x[1], y[0]:y[1]], dtype='uint64')) for z_idx in range(z[0],z[1])])
    rmt.create_cutout(chan, 0, x, y, z, np.ascontiguousarray(data_cut))
    
print('ok')


  0%|          | 8/3248 [00:08<58:19,  1.08s/it]  
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 88%|████████▊ | 2864/3248 [29:03<03:53,  1.64it/s]